In [1]:
import glc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dir(glc)

In [2]:
ds = glc.LoadExampleData()
feat_df = ds.feat_table
feat_dicts = glc.FeatDicts(feat_df)

ground_truth = ds.ground_truth

ggm_df = ds.ggm
ggm = glc.GGM(ggm_df)

4858
All nodes
4849


In [ ]:
sns.set_style('whitegrid')
sns.scatterplot(x='rt', y='mz', data=feat_df, hue=np.log2(feat_df.iloc[:, 7:]+1).median(axis=1).values, palette='viridis')

In [ ]:
umap_emb_obj = glc.UMAPEmbedder(ggm.G, remove_outliers=True)

In [ ]:
sns.set_style('darkgrid')
fig, axs = plt.subplots(1, 2, figsize=(18, 6))
glc.UMAPPlots(umap_emb_obj).plot_feature_attribute(feat_dicts, attribute='rt', ax=axs[0])
glc.UMAPPlots(umap_emb_obj).plot_feature_attribute(feat_dicts, attribute='mz', vmax=1200, ax=axs[1])

In [ ]:
sns.set_style('whitegrid')
fig, axs = plt.subplots(2, 1, figsize=(6.5, 12))
glc.UMAPPlots(umap_emb_obj).plot_ground_truth(ground_truth, class_level='lm_mainclass', ax=axs[0])
glc.UMAPPlots(umap_emb_obj).plot_ground_truth(ground_truth, class_level='lm_subclass', ax=axs[1])

In [ ]:
glc.UMAPEmbStats(umap_emb_obj, ground_truth, class_level='lm_mainclass').get_stats()

In [ ]:
glc.UMAPEmbStats(umap_emb_obj, ground_truth, class_level='lm_subclass').get_stats()

In [3]:
peak_picker = glc.PickPeaks(ggm, feat_dicts, rt_delta=2, only_pos_pcor=False)
peak_picker.get_peak_df(n_thresh=20).sort_values(by='n_edges', ascending=False)

,mz,n_edges,width,mzmin,mzmax,sigma,density,mz_peak_id
11,1.003999,1782,0.012472,0.997870,1.010342,0.002693,5.288509,11
4,0.369873,663,0.045424,0.344589,0.390013,0.008553,0.690852,4
7,0.634732,580,0.044684,0.614741,0.659425,0.010486,0.610847,7
20,2.008627,401,0.015796,2.001444,2.017240,0.003657,0.974137,20
32,4.956601,323,0.010968,4.951242,4.962211,0.002382,0.975613,32
...,...,...,...,...,...,...,...,...
85,41.997411,21,0.015022,41.989359,42.004381,0.002960,0.046811,85
25,2.947366,21,0.006614,2.943292,2.949906,0.001079,0.064708,25
88,76.049955,20,0.008336,76.046212,76.054548,0.001950,0.057346,88
40,16.587868,20,0.009885,16.583656,16.593541,0.002409,0.074714,40


In [ ]:
glc.MzPeakLookup().identify_mz_diffs(peak_picker.get_peak_df(n_thresh=20))

In [4]:
esi_data = {
    'formula': [
        'H',
        'Na',
        'K',
        'NH4',
        'H2O',
        'C2H2',
        'H2'

    ], 
    'type_code': [
        'charged',
        'charged',
        'charged',
        'charged',
        'isf',
        'isf',
        'isf'
    ]
}
esi_df = pd.DataFrame(esi_data)



In [ ]:
feat2lmids = glc.map_ungrouped_feature_table(
    db= glc.load_lm_database(), 
    feat_dicts=feat_dicts,
    ion_mode='pos', 
    proton_only=False, 
    esi_df=esi_df,
    ppm_interval=10, 
    mz_peak_picker_obj=peak_picker, 
    c13_adjustment=True, 
    logp_filter_tresh=3
)

/Users/tomrix/Library/CloudStorage/OneDrive-ImperialCollegeLondon/projects/glc/src/glc/data_loading.py:62: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  lm_df = pd.read_csv(f)
/Users/tomrix/Library/CloudStorage/OneDrive-ImperialCollegeLondon/projects/glc/src/glc/ion_processor.py:174: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dfs, axis=0).reset_index(drop=True)
Adjusting for C13 isotope components: 100%|██████████| 1439/1439 [00:00<00:00, 5200.39it/s]


In [14]:
from typing import Dict, List, Tuple
import pandas as pd
import numpy as np
import networkx as nx
from collections import Counter
from tqdm import tqdm
from functools import lru_cache



class ClassPredictor:
    def __init__(
        self, 
        ggm, 
        feat_dicts, 
        node_ids: Dict[int, List[str]], 
        db_df: pd.DataFrame, 
        db_id_col: str, 
        class_level: str, 
        rt_thresh: float = 50, 
        feat_weight: float = 5
    ):
        """
        Initialize the ClassPredictor with parameters.

        Parameters:
            ggm: Instance of the GGM class containing graph partial coefficients.
            feat_dicts: Instance of the FeatDicts class to look up feature info.
            node_ids: Dictionary mapping node IDs to lists of database IDs.
            db_df: DataFrame containing the database information.
            db_id_col: Column name in db_df that contains the database IDs (e.g., LM_ID).
            class_level: The class level to predict (e.g., SUB_CLASS).
            rt_thresh: Retention time threshold (default: 50).
            feat_weight: Weight for the feature being classified (default: 5).
        """
        self.ggm = ggm
        self.feat_dicts = feat_dicts
        self.node_ids = node_ids
        self.db_df = db_df
        self.db_id_col = db_id_col
        self.class_level = class_level
        self.rt_thresh = rt_thresh
        self.feat_weight = feat_weight
        self.lmid2class = db_df.set_index(db_id_col)[class_level].to_dict()

    @lru_cache(maxsize=None)
    def get_neighbors(self, feature_id: int) -> List[int]:
        """
        Retrieve neighbors of a feature in the graph.

        Parameters:
            feature_id: Feature ID.

        Returns:
            List of neighbor IDs.
        """
        return list(nx.neighbors(self.ggm.G, feature_id))

    def _filter_neighbors(self, feature_id: int, neighbors: List[int]) -> Tuple[List[int], List[float]]:
        """
        Filter neighbors by retention time and calculate weights.

        Parameters:
            feature_id: Feature ID being scored.
            neighbors: List of neighbor IDs.

        Returns:
            Filtered neighbor IDs and corresponding weights.
        """
        rt = self.feat_dicts.rt[feature_id]
        filtered = [
            (node, self.ggm.pcor_dict[f"{feature_id}::{node}"])
            for node in neighbors
            if abs(self.feat_dicts.rt[node] - rt) <= self.rt_thresh
        ]
        return zip(*filtered) if filtered else ([], [])

    def score_node(self, feature_id: int) -> List[Tuple[str, float]]:
        """
        Score the class for a given feature ID using the GGM model.

        Parameters:
            feature_id: Feature ID to predict the class for.

        Returns:
            List of tuples with class and score.
        """
        try:
            neighbors = self.get_neighbors(feature_id)
            neighbors, weights = self._filter_neighbors(feature_id, neighbors)

            # Include the target feature with a boosted weight
            neighbors = [feature_id] + list(neighbors)
            weights = [max(weights, default=0) * self.feat_weight] + list(weights)

            # Count class scores
            counter = Counter()
            for node, weight in zip(neighbors, weights):
                categories = {self.lmid2class[lmid] for lmid in self.node_ids[node]}
                for category in categories:
                    counter[category] += weight

            # Second-hop scoring
            counter = self._second_hop_score(neighbors[1:], weights[1:], counter)
            return counter.most_common(10)
        except KeyError as e:
            raise KeyError(f"Feature ID {feature_id} not found: {e}")
        except Exception as e:
            raise RuntimeError(f"Error scoring feature {feature_id}: {e}")

    def _second_hop_score_old(self, neighbors: List[int], weights: List[float], counter: Counter) -> Counter:
        """
        Calculate scores for second-hop neighbors.

        Parameters:
            neighbors: First-hop neighbor IDs.
            weights: Weights of first-hop neighbors.
            counter: Counter object to update scores.

        Returns:
            Updated Counter object.
        """
        for feature_id, weight_1hop in zip(neighbors, weights):
            second_neighbors = self.get_neighbors(feature_id)
            second_neighbors, weights_2hop = self._filter_neighbors(feature_id, second_neighbors)
            for node, weight_2hop in zip(second_neighbors, weights_2hop):
                categories = {self.lmid2class[lmid] for lmid in self.node_ids[node]}
                for category in categories:
                    counter[category] += weight_1hop * weight_2hop
        return counter
    
    def _second_hop_score(self, neighbors: List[int], weights: List[float], counter: Counter) -> Counter:
    
        """
        Calculate scores for second-hop neighbors.
        Avoids duplicate scoring by using a dictionary to track maximum scores.

        Parameters:
            neighbors: First-hop neighbor IDs.
            weights: Weights of first-hop neighbors.
            counter: Counter object to update scores.

        Returns:
            Updated Counter object.
        """

        max_weights = {}
        for feature_id, weight_1hop in zip(neighbors, weights):
            second_neighbors = self.get_neighbors(feature_id)
            second_neighbors, weights_2hop = self._filter_neighbors(feature_id, second_neighbors)

            for node, weight_2hop in zip(second_neighbors, weights_2hop):
                score = weight_1hop * weight_2hop
                if node not in max_weights or score > max_weights[node]:
                    max_weights[node] = score
        
        for node, score in max_weights.items():
            categories = {self.lmid2class[lmid] for lmid in self.node_ids[node]}
            for category in categories:
                counter[category] += score
        return counter


    def predict_all(self) -> Dict[int, List[Tuple[str, float]]]:
        """
        Predict classes for all features in the graph.

        Returns:
            Dictionary with feature IDs as keys and predicted classes as values.
        """
        print('Test its updated')
        predictions = {}
        missing_feats = []

        for feature_id in tqdm(self.ggm.G.nodes, desc="Predicting feature classes"):
            try:
                predictions[feature_id] = self.score_node(feature_id)
                
                if predictions[feature_id] == []:
                    missing_feats.append(feature_id)

            except Exception:
                missing_feats.append(feature_id)

        

        # Apply label propagation for missing features
        nx.set_node_attributes(self.ggm.G, {node: preds[0][0] for node, preds in predictions.items() if preds}, "label")
        prop_labels = nx.algorithms.node_classification.harmonic_function(self.ggm.G)
        for feature_id in missing_feats:
            predictions[feature_id] = [(prop_labels[feature_id], np.nan)]

        return predictions


def convert2mainclass(lm_df: pd.DataFrame, predictions: Dict[int, List[Tuple[str, float]]]) -> Dict[int, List[Tuple[str, float]]]:
    mainclass_converter_dict = dict(zip(lm_df['SUB_CLASS'], lm_df['MAIN_CLASS']))
    output_dict = {}
    for key, value_list in predictions.items():
        unique_classes = set()
        output_dict[key] = [
            (mainclass_converter_dict[item], score)
            for item, score in value_list
            if mainclass_converter_dict.get(item) and mainclass_converter_dict[item] not in unique_classes
            and not unique_classes.add(mainclass_converter_dict[item])
        ]
    return output_dict

In [16]:
predictions = ClassPredictor(
    ggm=ggm, 
    feat_dicts=feat_dicts, 
    node_ids=feat2lmids, 
    db_df=glc.load_lm_database(),
    db_id_col='LM_ID', 
    class_level='SUB_CLASS', 
    rt_thresh=50, 
    feat_weight=5
).predict_all()

/Users/tomrix/Library/CloudStorage/OneDrive-ImperialCollegeLondon/projects/glc/src/glc/data_loading.py:62: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  lm_df = pd.read_csv(f)


Test its updated


Predicting feature classes: 100%|██████████| 4849/4849 [00:36<00:00, 131.76it/s]


In [20]:
predictions[258]

[('Monoacylglycerophosphates [GP1005]', np.float64(0.4780653875817667)),
 ('Halogenated fatty acids [FA0109]', np.float64(0.3433830967397812)),
 ('Alkyl-dihydroxyacetonephosphates [GP2202]',
  np.float64(0.34319558196865785)),
 ('Monoacyl cyclic glycerophosphatidic acids [GP2501]',
  np.float64(0.34288076455755684)),
 ('C40 isoprenoids (tetraterpenes) [PR0107]', np.float64(0.3111544107122577)),
 ('1Z-alkenylglycerophosphates [GP1007]', np.float64(0.3047394591566729)),
 ('Other Fatty Acyls [FA00]', np.float64(0.3047394591566729)),
 ('Monoacylglycerophosphocholines [GP0105]', np.float64(0.15819292145543845)),
 ('Chalcones and dihydrochalcones [PK1212]', np.float64(0.13252728652923887)),
 ('Monoacylglycerophosphoethanolamines [GP0205]',
  np.float64(0.11994359683113913))]

In [13]:
feat2lmids[924]

['LMFA07090147']

In [7]:
feat2lmids

{1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 32: [],
 33: [],
 34: [],
 35: [],
 36: [],
 37: [],
 38: [],
 39: [],
 40: [],
 41: [],
 42: [],
 43: [],
 44: [],
 45: [],
 46: [],
 47: [],
 48: [],
 49: [],
 50: [],
 51: [],
 52: [],
 53: [],
 54: [],
 55: [],
 56: [],
 57: [],
 58: [],
 59: ['LMFA01020209',
  'LMFA01030140',
  'LMFA01030141',
  'LMFA01030142',
  'LMFA01030143',
  'LMFA01030144',
  'LMFA01030145',
  'LMFA01030146',
  'LMFA01030147',
  'LMFA01030148',
  'LMFA01030151',
  'LMFA01030152',
  'LMFA01030153',
  'LMFA01030154',
  'LMFA01030155',
  'LMFA01030156',
  'LMFA01030339',
  'LMFA01030341',
  'LMFA01030342',
  'LMFA01030343',
  'LMFA01030344',
  'LMFA01030345',
  'LMFA01030346',
  'LMFA01030347',
  'LMFA01030348',
  'LMFA01030349',
  'LMFA01030350

In [ ]:
feat_mapper = glc.FeatMapper('pos', esi_df, peak_picker.get_peak_df(n_thresh=20), peak_picker.classify_edges(), feat_dicts, ppm_interval=10)
feat_mapper.compute_multiple(feat_dicts.mz.keys())
# Convert feat_mapper.results to a DataFrame for fast querying
features_data = []
for feat, data in feat_mapper.results.items():
    for adduct, mass_data in data.items():
        features_data.append({
            'feat': feat,
            'ppm_min': mass_data['ppm_range'][0],
            'ppm_max': mass_data['ppm_range'][1]
        })
features_df = pd.DataFrame(features_data)

In [ ]:
from collections import defaultdict
from tqdm import tqdm

lm_df = glc.load_lm_database()
lm_ids = lm_df['LM_ID'].values
exact_masses = lm_df['EXACT_MASS'].values

# Iterate over each lipid mass
feat2lmids = defaultdict(list)
for lm_id, exact_mass in tqdm(zip(lm_ids, exact_masses), total=len(lm_ids)):
    # Vectorized range check for the current exact_mass
    mask = (features_df['ppm_min'] <= exact_mass) & (exact_mass <= features_df['ppm_max'])
    # Get matching features
    matching_feats = features_df.loc[mask, 'feat'].values
    # Append lm_id to each matching feature
    for feat in matching_feats:
        feat2lmids[feat].append(lm_id)



In [ ]:
# benchmark

print('test 1')
print(feat2lmids[249])
print(feat2lmids[251]) # IDs from 1 should be added here

print('test 2')
print(feat2lmids[259]) # and IDs from 2 should not be here
print(feat2lmids[262]) # Ids from 2 should remain here

print('test 3')
print(feat2lmids[269]) 
print(feat2lmids[271]) # IDs from 1 should be added here and importantly 054 should remain

In [ ]:
# Perfect this class works and its been checked to be identical to previous implementation
# When put in package make sure to copy the dict so its not overwritten

import networkx as nx
import pandas as pd
from molmass import Formula
from tqdm import tqdm


class AdjustForC13:
    def __init__(self, feat2lmids, mz_rule_df, edges_clf):
        self.feat2lmids = feat2lmids
        self.mz_rule_df = mz_rule_df
        self.edges_clf = edges_clf
        self.iso_delta = Formula('13C').isotope.mass - Formula('12C').isotope.mass

    def _get_isotope_mz_peak_id(self):
        intervals = pd.IntervalIndex.from_arrays(
            self.mz_rule_df['mzmin'],
            self.mz_rule_df['mzmax'],
            closed='both'
        )
        matches = self.mz_rule_df.loc[intervals.get_loc(self.iso_delta)]

        if isinstance(matches, pd.DataFrame):
            return int(matches.loc[matches['n_edges'].idxmax(), 'mz_peak_id'])
        else:
            return int(matches['mz_peak_id'])

    def _get_components(self, df):
        G = nx.from_pandas_edgelist(df, 'feature_out', 'feature_in')
        return list(nx.connected_components(G))

    def run(self):
        mz_peak_id = self._get_isotope_mz_peak_id()
        iso_edges = self.edges_clf[self.edges_clf['mz_peak_id'] == mz_peak_id]

        components = self._get_components(iso_edges)

        for comp in tqdm(components, desc='Adjusting for C13 isotope components'):
            comp_df = iso_edges[
                iso_edges['feature_out'].isin(comp) |
                iso_edges['feature_in'].isin(comp)
            ]


            # FIX: use .loc not .iloc
            idx = comp_df['out_mz'].idxmin() # get the row with the smallest m/z
            primary_row = comp_df.loc[idx]

            primary_feat = int(primary_row['feature_out'])
            primary_struct = self.feat2lmids[primary_feat]

            for feat in comp:
                if feat != primary_feat:
                    self.feat2lmids[feat].extend(primary_struct)

        return self.feat2lmids


In [ ]:
feat2lmids_adjusted = glc.AdjustForC13(feat2lmids, peak_picker.get_peak_df(n_thresh=20), peak_picker.classify_edges()).run()

In [ ]:
# benchmark

print('test 1')
print(feat2lmids_adjusted[249])
print(feat2lmids_adjusted[251]) # IDs from 1 should be added here

print('test 2')
print(feat2lmids_adjusted[259]) # and IDs from 2 should not be here
print(feat2lmids_adjusted[262]) # Ids from 2 should remain here

print('test 3')
print(feat2lmids_adjusted[269]) 
print(feat2lmids_adjusted[271]) # IDs from 1 should be added here and importantly 054 should remain

In [ ]:
feat2lmids_adjusted[258]

In [ ]:
feat2lmids, logpdf = glc.filter_by_logp(
    feat2lmids=feat2lmids_adjusted,
    lm_df=lm_df,
    n_stds=2,
    feat_dicts=feat_dicts,
    return_df=True
)

In [ ]:
import hashlib
import json

feat2lmids_adjusted_n = {}
for feat, lmids in feat2lmids_adjusted.items():
    feat2lmids_adjusted_n[int(feat)] = sorted(lmids)

def hash_dict(d: dict) -> str:
    """Return a deterministic SHA-256 hash of a Python dict."""
    payload = json.dumps(d, sort_keys=True, default=str).encode()
    return hashlib.sha256(payload).hexdigest()
    
hash_dict(feat2lmids_adjusted_n)

In [9]:
import pickle
import os
with open(os.path.expanduser('~/Downloads/feat2lmids_test.pkl'), 'wb') as f:
    pickle.dump(feat2lmids, f)

In [17]:
import pickle
import os
with open(os.path.expanduser('~/Downloads/predictions_test.pkl'), 'wb') as f:
    pickle.dump(predictions, f)

In [ ]:
glc.FeatMapper('pos', pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), feat_dicts, ppm_interval=10)

In [ ]:
from molmass import Formula
Formula('H').monoisotopic_mass

In [ ]:
1 -- 1

In [ ]:
-1 --1 